In [2]:
import os
import time
import azure.cognitiveservices.speech as speechsdk
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
def recognize_from_file(filename: str, language: str="en-GB"):
    speech_config = speechsdk.SpeechConfig(subscription=os.getenv('SPEECH_KEY'), endpoint="https://uksouth.api.cognitive.microsoft.com")
    speech_config.speech_recognition_language=language
    speech_config.set_property(property_id=speechsdk.PropertyId.SpeechServiceResponse_DiarizeIntermediateResults, value='true')

    audio_config = speechsdk.audio.AudioConfig(filename=filename)
    conversation_transcriber = speechsdk.transcription.ConversationTranscriber(speech_config=speech_config, audio_config=audio_config)

    transcribing_stop = False

    def conversation_transcriber_session_started_cb(evt: speechsdk.SessionEventArgs):
        print('SessionStarted event')

    def conversation_transcriber_session_stopped_cb(evt: speechsdk.SessionEventArgs):
        print('SessionStopped event')
        
    def conversation_transcriber_recognition_canceled_cb(evt: speechsdk.SessionEventArgs):
        print('Canceled event')

    final_transcription = []

    def conversation_transcriber_transcribed_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        print('\nTRANSCRIBED:')
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            print('\tText={}'.format(evt.result.text))
            print('\tSpeaker ID={}\n'.format(evt.result.speaker_id))
            #final_transcription.append({"text":evt.result.text, "speaker_id":evt.result.speaker_id})
            final_transcription.append(f"{evt.result.speaker_id}: {evt.result.text}")
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print('\tNOMATCH: Speech could not be TRANSCRIBED: {}'.format(evt.result.no_match_details))

    def stop_cb(evt: speechsdk.SessionEventArgs):
        #"""callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal transcribing_stop
        transcribing_stop = True

    # Connect callbacks to the events fired by the conversation transcriber
    conversation_transcriber.transcribed.connect(conversation_transcriber_transcribed_cb)
    conversation_transcriber.session_started.connect(conversation_transcriber_session_started_cb)
    conversation_transcriber.session_stopped.connect(conversation_transcriber_session_stopped_cb)
    conversation_transcriber.canceled.connect(conversation_transcriber_recognition_canceled_cb)
    # stop transcribing on either session stopped or canceled events
    conversation_transcriber.session_stopped.connect(stop_cb)
    conversation_transcriber.canceled.connect(stop_cb)

    conversation_transcriber.start_transcribing_async()

    # Waits for completion.
    while not transcribing_stop:
        time.sleep(.5)

    conversation_transcriber.stop_transcribing_async()

    return "\n".join(final_transcription)

In [15]:
try:
    result = recognize_from_file(r"C:\Users\Beth\Documents\Bupa\Real_Time_STT\audio\the_room.wav", language="en-US")
except Exception as err:
    print("Encountered exception. {}".format(err))

SessionStarted event

TRANSCRIBED:
	Text=Hi can I help you?
	Speaker ID=Guest-1


TRANSCRIBED:
	Text=Yeah, can I have a dozen red roses, please?
	Speaker ID=Guest-2


TRANSCRIBED:
	Text=Oh, hi, Johnny. I didn't know it was you.
	Speaker ID=Guest-1


TRANSCRIBED:
	Text=Here you go.
	Speaker ID=Guest-1


TRANSCRIBED:
	Text=That's me.
	Speaker ID=Guest-2


TRANSCRIBED:
	Text=How much?
	Speaker ID=Guest-2


TRANSCRIBED:
	Text=Is it? It'll be $18.
	Speaker ID=Guest-1


TRANSCRIBED:
	Text=Keep the change.
	Speaker ID=Guest-2


TRANSCRIBED:
	Text=Hi, doggie. You're my favorite customer. Thanks a lot. Bye. Bye. Bye.
	Speaker ID=Guest-1

Canceled event
CLOSING on ConversationTranscriptionCanceledEventArgs(session_id=4c4510740e4a4c289aeab7449edf88f5, result=ConversationTranscriptionResult(result_id=ae7db9c158ba48dd9eacc2ffeeaada31, speaker_id=, text=, reason=ResultReason.Canceled))
SessionStopped event
CLOSING on SessionEventArgs(session_id=4c4510740e4a4c289aeab7449edf88f5)


In [16]:
print(result)

Guest-1: Hi can I help you?
Guest-2: Yeah, can I have a dozen red roses, please?
Guest-1: Oh, hi, Johnny. I didn't know it was you.
Guest-1: Here you go.
Guest-2: That's me.
Guest-2: How much?
Guest-1: Is it? It'll be $18.
Guest-2: Keep the change.
Guest-1: Hi, doggie. You're my favorite customer. Thanks a lot. Bye. Bye. Bye.


In [21]:
try:
    result = recognize_from_file(r"C:\Users\Beth\Documents\Bupa\Real_Time_STT\audio\jackie_weaver.wav", language="en-GB")
except Exception as err:
    print("Encountered exception. {}".format(err))

SessionStarted event

TRANSCRIBED:
	Text=So, Chairman, will be it late, shall we get this meeting started? We'll.
	Speaker ID=Guest-1


TRANSCRIBED:
	Text=Start the meeting and I want to repeat what I said at the beginning of the last meeting. This meeting has not been called according to the law. The law has been broken.
	Speaker ID=Guest-2


TRANSCRIBED:
	Text=He has been properly.
	Speaker ID=Guest-1


TRANSCRIBED:
	Text=Will you please let the please. You have no authority here, Jackie Weaver. No authority at all.
	Speaker ID=Guest-2


TRANSCRIBED:
	Text=Don't don't just kicked him out.
	Speaker ID=Guest-3


TRANSCRIBED:
	Text=This is a meeting called by two councillors.
	Speaker ID=Guest-1


TRANSCRIBED:
	Text=Illegally, they now elect a chairman.
	Speaker ID=Guest-1


TRANSCRIBED:
	Text=The carpet for the vice chair. Vice chairs. Here. I take charge, read the standing orders, read them and understand them.
	Speaker ID=Guest-3


TRANSCRIBED:
	Text=Where's the chairman gone?
	Speak

In [22]:
print(result)

Guest-1: So, Chairman, will be it late, shall we get this meeting started? We'll.
Guest-2: Start the meeting and I want to repeat what I said at the beginning of the last meeting. This meeting has not been called according to the law. The law has been broken.
Guest-1: He has been properly.
Guest-2: Will you please let the please. You have no authority here, Jackie Weaver. No authority at all.
Guest-3: Don't don't just kicked him out.
Guest-1: This is a meeting called by two councillors.
Guest-1: Illegally, they now elect a chairman.
Guest-3: The carpet for the vice chair. Vice chairs. Here. I take charge, read the standing orders, read them and understand them.
Guest-2: Where's the chairman gone?
Guest-1: I'd like to elect a chairman for this meeting.
Guest-4: You don't have to elect a chairman. Does the chairman already installed the chairman of the council?
Guest-1: Councillor Berkel, we've been through this. What?
Unknown: Are you talking about? You don't know what you're talking ab